# GCN for DAVIS 2016

In this notebook, a custom [PyTorch Geometric](https://rusty1s.github.io/pytorch_geometric/build/html/index.html) [InMemoryDataset](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/in_memory_dataset.html#InMemoryDataset) for the DAVIS 2016 dataset is created. The implementation is based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/create_dataset.html). The dataset is then used to train a simple GCN network as a first evaluation based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/introduction.html#learning-methods-on-graphs).

The dataset consists of single PyTorch Geometric [Data](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/data.html#Data) objects which model a single graph with various attributes. For this dataset, a graph for each contour is created. Hereby, each node of the graph represents one contour point. The feature of each node is the OSVOS feature vector from the next frame at this point. Each node is connected to its K nearest neighbours. The feature of each edge is the distance between the nodes it connects. The targets of each node is the translation it undergoes from the current to the next frame.

## Imports

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

from pg_datasets.davis_2016 import DAVIS2016

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Paths & Constants

In [2]:
PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH = 'pg_datasets/DAVIS_2016'
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
IMAGES_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'

LAYER = 9
K = 32

SKIP_SEQUENCES = ['bmx-trees', 'bus', 'cows', 'dog-agility', 'horsejump-high', 
                  'horsejump-low', 'kite-walk', 'lucia', 'libby', 'motorbike',
                  'paragliding', 'rhino', 'scooter-gray', 'swing']

TRAIN_SEQUENCES = ['bear', 'bmx-bumps', 'boat', 'breakdance-flare', 'bus', 
                   'car-turn', 'dance-jump', 'dog-agility', 'drift-turn', 
                   'elephant', 'flamingo', 'hike', 'hockey', 'horsejump-low', 
                   'kite-walk', 'lucia', 'mallard-fly', 'mallard-water', 
                   'motocross-bumps', 'motorbike', 'paragliding', 'rhino', 
                   'rollerblade', 'scooter-gray', 'soccerball', 'stroller',
                   'surf', 'swing', 'tennis', 'train']

VAL_SEQUENCES = ['blackswan', 'bmx-trees', 'breakdance', 'camel', 'car-roundabout',
                 'car-shadow', 'cows', 'dance-twirl', 'dog', 'drift-chicane', 
                 'drift-straight', 'goat', 'horsejump-high', 'kite-surf', 'libby', 
                 'motocross-jump', 'paragliding-launch', 'parkour', 'scooter-black', 
                 'soapbox']

BATCH_SIZE = 16

## Dataset

In [3]:
train = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                  CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                  LAYER, K, 
                  SKIP_SEQUENCES, TRAIN_SEQUENCES, VAL_SEQUENCES,
                  train=True)

Processing...
#0: bear
Start online training...
Finished online training...
Create new OSVOS model...
Constructing OSVOS architecture..
Initializing weights..
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x.dtype=torch.float64
edge_index.dtype=torch.float64
edge_attr.dtype=torch.float64
y.dtype=torch.float64
x

KeyboardInterrupt: 

In [ ]:
val = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                LAYER, K, 
                SKIP_SEQUENCES, TRAIN_SEQUENCES, VAL_SEQUENCES,
                train=False)

In [ ]:
data = train[0]
for key, item in data:
    print(key, type(item))

print(data)
print(data.num_features)
print(data.num_nodes)
y = data.y
print(y.shape)
y = y.flatten()
print(y.shape)

In [ ]:
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)

## Simple GCN

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()
        
        self.conv1 = GCNConv(in_channels, 256)
        self.conv2 = GCNConv(256, 512)
        self.conv3 = GCNConv(512, 1024)
        
        self.lin1 = nn.Linear(1024, 512)
        self.lin2 = nn.Linear(512, out_channels)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.lin1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)   
        
        x = self.lin2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)    
        
        return x

In [ ]:
def train_net(n_epochs):

    # prepare the net for training
    model.train()
    running_loss = 0.0
    
    # loop over the dataset multiple times
    for epoch in range(n_epochs):
        
        # train on batches of data, assumes you already have train_loader
        for batch_i, data in enumerate(train_loader):

            # convert variables to floats for regression loss
            # data = data.type(torch.FloatTensor)
            
            data = data.to(device)

            # forward pass to get outputs
            out = model(data)

            # calculate the loss between predicted and target keypoints
            loss = criterion(out, data.y.flatten)

            # zero the parameter (weight) gradients
            optimizer.zero_grad()

            # backward pass to calculate the weight gradients
            loss.backward()

            # update the weights
            optimizer.step()
            
            running_loss += loss.item()

            # print loss statistics
            # to convert loss into a scalar and add it to the running_loss, use .item()
            if batch_i % 10 == 9:    # print every 10 batches
                print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1,
                                                                   batch_i+1,
                                                                   running_loss / (len(train_loader)*epoch+batch_i)))
            
            #return images, key_pts, output_pts

    print('Finished Training')

    
# Load model and run the solver
model = Net(in_channels=train[0].num_features, 
            out_channels=train[0].num_nodes * 2)
model.float()
print(model)
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9, weight_decay=1e-6, nesterov=True)
train_net(n_epochs=50)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net(in_channels=train[0].num_features, 
            out_channels=train[0].num_nodes * 2).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))